# 📦 Chapter 7 Exercise: Roughness Calculation

One very useful feature in 3D processing is "Roughness" (or Surface Variation). It basically tells you how "bumpy" a surface is. It is calculated as the distance of points from their best-fit plane.

**Your Goal:**
1.  Load `features_verviers.ply` (or a subset).
2.  For each point, find its K-Nearest Neighbors (K=10).
3.  Fit a plane to these neighbors (using PCA/SVD, the normal is likely the 3rd eigenvector).
4.  **Simpler Proxy**: The eigenvalue ratio $λ_3 / (λ_1 + λ_2 + λ_3)$ is often used as a measure of surface variation (Change of Curvature).
5.  Compute this `Surface_Variation` feature and visualize it.

In [ ]:
import numpy as np
import pyvista as pv
from scipy.spatial import KDTree

# 1. Load Data
pcd = pv.read("../DATA/features_verviers.ply")

# Work on a subset for speed
n_points = 2000
indices_subset = np.random.choice(pcd.n_points, n_points, replace=False)
points_subset = pcd.points[indices_subset]

# 2. Build Tree
tree = KDTree(pcd.points)

# TODO: Query neighbors for the subset
k = 10
dists, indices = tree.query(points_subset, k=k)

# Array to store results
curvature = np.zeros(n_points)

for i in range(n_points):
    neighbors = pcd.points[indices[i]]
    
    # TODO: Compute PCA on neighbors
    mean = np.mean(neighbors, axis=0)
    centered = neighbors - mean
    cov = np.cov(centered, rowvar=False)
    vals, _ = np.linalg.eig(cov)
    
    # Sort eigenvalues
    vals = np.sort(vals)[::-1]
    l1, l2, l3 = vals
    
    # TODO: Calculate Curvature Proxy
    # curvature[i] = ...
    # Formula: l3 / (l1 + l2 + l3)
    if np.sum(vals) > 0:
        curvature[i] = l3 / np.sum(vals)

# Visualize
pcd_sub = pcd.extract_points(indices_subset)
pcd_sub['Curvature'] = curvature
pcd_sub.plot(scalars='Curvature', cmap='jet', point_size=8, render_points_as_spheres=True)